In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
# Base URL for pagination
base_url = "https://www.supermarches.ca/pages/Default.asp"

# Parameters for pagination
params = {
    "page": 1,        # Start from page 1
}

# Headers to simulate a browser request
headers = {"User-Agent": "Mozilla/5.0"}

# List to store extracted food items
food_data = []


In [4]:
while params["page"] <= 250:
    # Fetch the webpage
    response = requests.get(base_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all <tr> rows
    rows = soup.find_all("tr")
    items_found = False  # Track if items were found on the page

    for row in rows:
        cols = row.find_all("td")
        if len(cols) == 7:  # Ensure valid item structure
            a_tag = cols[1].find("a", class_="dslink")
            
            if a_tag:
                raw_text = a_tag.get_text(separator="\n").strip()
                lines = [line.strip() for line in raw_text.splitlines() if line.strip()]
                
                item_name = lines[0]  # First line is the item name
                extra_info = lines[1] if len(lines) > 1 else ""  # Second line if available
            else:
                raw_text = cols[1].get_text(separator="\n").strip()
                lines = [line.strip() for line in raw_text.splitlines() if line.strip()]
                
                item_name = lines[0]
                extra_info = lines[1] if len(lines) > 1 else ""

            brand = cols[3].get_text(strip=True)

            # Only add extra_info if it's not the same as the brand
            if extra_info and extra_info.lower() != brand.lower():
                item_name = f"{item_name} {extra_info}"

            # Handle multiple prices in the same cell
            price_cell = cols[4].get_text(separator="\n").strip()
            prices = [p.strip() for p in price_cell.splitlines() if p.strip()]  # Split by <br> tags

            store_info = cols[6].get_text(strip=True)

            # Prioritize "/ lb" prices if both "/ lb" and "/ kg" are present
            selected_price = None
            for price in prices:
                if "/ lb" in price:
                    selected_price = price
                    break  # Prioritize the first "/ lb" price
            if not selected_price:  # If no "/ lb" price, use the first valid price
                selected_price = prices[0] if prices else None

            # Validate the selected price
            if selected_price:
                valid_price = (
                    any(char.isdigit() for char in selected_price) and 
                    any(unit in selected_price for unit in ["/ lb", "/ kg", "/ ch.", "/ unité", "$", "/ caisse", "/ sac"])
                )

                if valid_price:
                    food_data.append([item_name, brand, selected_price, store_info])
                    items_found = True
                else:
                    print(f"Skipping invalid price format: {selected_price}")  # Debugging output

    print(f"Scraped Page {params['page']}...")

    if not items_found:
        break  # Stop if no items found on this page

    params["page"] += 1


Skipping invalid price format: 3 / 6.00
Skipping invalid price format: 3 / 6.00
Skipping invalid price format: 3 / 6.00
Skipping invalid price format: Toutes les circulaires
Scraped Page 1...
Skipping invalid price format: 2 / 9.00
Skipping invalid price format: Toutes les circulaires
Scraped Page 2...
Skipping invalid price format: 2 / 10.00
Skipping invalid price format: 2 / 7.00
Skipping invalid price format: 2 / 5.00
Skipping invalid price format: Toutes les circulaires
Scraped Page 3...
Skipping invalid price format: 2 / 9.00
Skipping invalid price format: Toutes les circulaires
Scraped Page 4...
Skipping invalid price format: 2 / 10.00
Skipping invalid price format: Toutes les circulaires
Scraped Page 5...
Skipping invalid price format: Toutes les circulaires
Scraped Page 6...
Skipping invalid price format: Toutes les circulaires
Scraped Page 7...
Skipping invalid price format: 2 / 7.00
Skipping invalid price format: 2 / 10.00
Skipping invalid price format: Toutes les circulaires

In [10]:
df = pd.DataFrame(food_data, columns=["name", "brand", "price", "Store"])

In [11]:
df

,name,brand,price,Store
0,Tartare de bœuf surgelé,La Cage,11.99 / ch.,METROCirc.-Mag.
1,Gaufres surgelées Eggo,Kellogg's,5.99 / ch.,METROCirc.-Mag.
2,Coquille aux fruits de mer-repas surgelée Voir...,---,6.99 / ch.,METROCirc.-Mag.
3,Mets chinois surgelés,Wong Wing,6.99 / ch.,SUPER CCirc.-Mag.
4,Dumplings surgelés,Wong Wing,6.99 / ch.,SUPER CCirc.-Mag.
...,...,...,...,...
1996,Souvlakis de poulet frais,Marc Angelo,12.99 / ch.,IGACirc.-Mag.
1997,Dinde effilochée,Maple Leaf,7.99 / ch.,PHARMAPRIXCirc.-Mag.
1998,Alouettes de poulet farcies Metrogo!,---,9.99 / lb,METROCirc.-Mag.
1999,Poulet effiloché Apprêtez et savourez,Maple Leaf,7.99 / ch.,PHARMAPRIXCirc.-Mag.


In [12]:
# Remove 'Circ.-Mag.' from 'Store Info' and rename to 'Store'
df["store"] = df["Store"].str.replace('Circ.-Mag.', '', regex=True)
df.drop(columns=["Store"], inplace=True)

# Keep original price string in 'Price' column
df["price"] = df["price"].str.replace(',', '.', regex=True)  # Convert commas to decimal points

# Extract numeric price for calculations
df["nominal"] = df["price"].str.extract(r'(\d+\.\d+)')  # Extract only the numeric part

# Filter out invalid data
df["nominal"] = df["nominal"].astype(float)  # Convert nominal price to float
df = df[df["nominal"] < 100]  # Remove any abnormally high prices


# For items appearing with both lb and kg prices, keep only the lb version
df = df.sort_values(by=["name", "brand", "store"])  # Ensures '/ lb' appears before '/ kg'
df = df.drop_duplicates(subset=["name", "brand", "store"], keep="first")  # Keeps '/ lb' if both exist

# Keep only the lowest price per (Item Name, Price, Brand, Store)
df = df.loc[df.groupby(['name', 'price', 'brand', 'store'])['nominal'].idxmin()].reset_index(drop=True)

In [13]:
df

,name,brand,price,store,nominal
0,Acétaminophène 325 mg,Tylenol,18.99 / ch.,BRUNET,18.99
1,Acétaminophène 500 mg,Tylenol,10.99 / ch.,PHARMAPRIX,10.99
2,Acétaminophène 500 mg,Tylenol,8.99 / ch.,BRUNET,8.99
3,Acétaminophène Complet,Tylenol,12.29 / ch.,BRUNET,12.29
4,Acétaminophène Complet,Tylenol,15.99 / ch.,PHARMAPRIX,15.99
...,...,...,...,...,...
1887,Échalotes françaises,Attitude Fraîche,4.99 / ch.,SUPER C,4.99
1888,Écran solaire,Neutrogena,12.99 / ch.,UNIPRIX,12.99
1889,Édulcorant sans calories,Sugar Twin,5.99 / ch.,FAMILIPRIX,5.99
1890,Émincé de poitrine de dinde,Selection,7.99 / ch.,SUPER C,7.99


In [14]:
df.to_csv("supermarches.csv", index=False)